In [ ]:
library(tidyverse)
library(lubridate)
library(forecast)
library(car)
library(boot)
library(zoo)
library(factoextra)
cities <- c(
  "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
  "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"
)
set.seed(5100)

In [ ]:

pm25_raw <- read_csv("../data/daily_88101_2023.csv")
o3_raw   <- read_csv("../data/daily_44201_2023.csv")
no2_raw  <- read_csv("../data/daily_42602_2023.csv")

clean_pollutant <- function(df, pollutant_name) {
  df %>%
    filter(`City Name` %in% cities) %>%          # filter ROWS by city name
    mutate(Date = ymd(`Date Local`)) %>%         # convert date
    select(`City Name`, Date, `Arithmetic Mean`) %>%  # select correct columns
    rename(
      City = `City Name`,
      !!pollutant_name := `Arithmetic Mean`
    )
} 

pm25 <- clean_pollutant(pm25_raw, "PM25")
o3   <- clean_pollutant(o3_raw,   "O3")
no2  <- clean_pollutant(no2_raw,  "NO2")

air_df <- pm25 %>%
  inner_join(o3, by = c("City", "Date")) %>%
  inner_join(no2, by = c("City", "Date"))

# There are duplicate rows as the indicators have multiple monitoring sites per city. 
air_df_clean <- air_df %>%
  group_by(City, Date) %>%
  summarise(
    PM25 = mean(PM25, na.rm = TRUE),
    O3   = mean(O3,   na.rm = TRUE),
    NO2  = mean(NO2,  na.rm = TRUE),
    .groups = "drop"
  ) %>%
  arrange(City, Date)

Rows: 847057 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (16): State Code, County Code, Site Num, Datum, Parameter Name, Sample ...
dbl  (11): Parameter Code, POC, Latitude, Longitude, Observation Count, Obse...
date  (2): Date Local, Date of Last Change

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 388292 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (17): State Code, County Code, Site Num, Datum, Parameter Name, Sample ...
dbl  (10): Parameter Code, POC, Latitude, Longitude, Observation Count, Obse...
date  (2): Date Local, Date of Last Change

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 156472 Columns: 29
── Column specifi

[1] "2023-01-01" "2023-12-31"

# A tibble: 6 × 5
  City    Date        PM25     O3   NO2
  <chr>   <date>     <dbl>  <dbl> <dbl>
1 Chicago 2023-03-01  9.17 0.0264 15.8 
2 Chicago 2023-03-02  2.8  0.0365  9.47
3 Chicago 2023-03-04 16.1  0.0211 19.6 
4 Chicago 2023-03-07  3.15 0.0381  6.37
5 Chicago 2023-03-10  3.3  0.0339 10.0 
6 Chicago 2023-03-13  4.18 0.0323 11.9 



Data contains 3,367 rows instead of 3,650 because some cities have missing days, some monitoring stations fail to record, etc.